# 批量归一化 (Batch Normalization)

## 核心思想

批量归一化（Ioffe & Szegedy, 2015）通过对每一层的输入进行归一化，解决深度网络训练中的**内部协变量偏移 (Internal Covariate Shift)** 问题。

## 数学原理

对于 mini-batch B = {x₁, x₂, ..., xₘ}：

1. **计算均值**: μ_B = (1/m) Σxᵢ
2. **计算方差**: σ²_B = (1/m) Σ(xᵢ - μ_B)²
3. **归一化**: x̂ᵢ = (xᵢ - μ_B) / √(σ²_B + ε)
4. **缩放平移**: yᵢ = γx̂ᵢ + β

其中 γ（scale）和 β（shift）是可学习参数，ε 是防止除零的小常数。

## 主要优势

| 优势 | 说明 |
|------|------|
| 加速训练 | 允许使用更大的学习率 |
| 稳定训练 | 减少梯度消失/爆炸 |
| 正则化效果 | 类似轻度 Dropout |
| 降低初始化敏感度 | 对权重初始化不那么敏感 |

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow 版本: {tf.__version__}")

## 1. 基本用法：激活函数后使用 BN

传统方式是在激活函数之后应用 BN。

In [ ]:
# 构建带 BN 的模型（标准方式：激活函数后使用 BN）
model_bn_after = keras.models.Sequential([
    # 输入层：展平 28x28 图像
    keras.layers.Flatten(input_shape=(28, 28)),
    
    # 第一个 BN 层：归一化输入数据
    keras.layers.BatchNormalization(),
    
    # 第一个隐藏层
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    
    # 第二个隐藏层
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    
    # 输出层（分类任务）
    keras.layers.Dense(10, activation='softmax')
])

model_bn_after.summary()

In [ ]:
# 查看第一个 BN 层的参数
bn_layer = model_bn_after.layers[1]  # 第一个 BN 层

print("BatchNormalization 层参数:")
print("="*50)

for var in bn_layer.variables:
    trainable_status = "可训练" if var.trainable else "不可训练"
    print(f"{var.name:40s} | {trainable_status}")

print("\n参数说明:")
print("- gamma: 缩放因子（可训练）")
print("- beta: 偏移量（可训练）")
print("- moving_mean: 滑动均值（推理时使用，不可训练）")
print("- moving_variance: 滑动方差（推理时使用，不可训练）")

## 2. 优化方式：激活函数前使用 BN

论文原作者建议在激活函数**之前**应用 BN，这样可以：
- 归一化线性组合的输出
- 避免 Dense 层的 bias 与 BN 的 beta 参数冗余

In [ ]:
# 构建优化版本：BN 在激活函数之前
model_bn_before = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.BatchNormalization(),
    
    # Dense 层不使用 bias（BN 的 beta 会替代 bias 的作用）
    keras.layers.Dense(300, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),  # 激活函数单独作为一层
    
    keras.layers.Dense(300, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    
    keras.layers.Dense(10, activation='softmax')
])

model_bn_before.summary()

# 对比参数量
print(f"\n参数量对比:")
print(f"BN 在激活后: {model_bn_after.count_params():,} 参数")
print(f"BN 在激活前: {model_bn_before.count_params():,} 参数")

## 3. 训练与推理的区别

BN 在训练和推理阶段有不同的行为：

| 阶段 | 均值/方差来源 | 说明 |
|------|---------------|------|
| 训练 | 当前 mini-batch | 实时计算，同时更新滑动统计量 |
| 推理 | 滑动均值/方差 | 使用训练期间累积的统计量 |

In [ ]:
# 演示 BN 在训练和推理时的不同行为
def demonstrate_bn_behavior():
    """
    演示 BatchNormalization 在训练和推理阶段的不同行为
    """
    # 创建简单的 BN 层
    bn = keras.layers.BatchNormalization()
    
    # 模拟输入数据
    x_train = tf.random.normal((32, 10))  # batch_size=32, features=10
    x_test = tf.random.normal((8, 10))    # 测试数据
    
    # 训练模式
    output_train = bn(x_train, training=True)
    print("训练模式:")
    print(f"  输入均值: {tf.reduce_mean(x_train).numpy():.4f}")
    print(f"  输出均值: {tf.reduce_mean(output_train).numpy():.4f}")
    print(f"  输入标准差: {tf.math.reduce_std(x_train).numpy():.4f}")
    print(f"  输出标准差: {tf.math.reduce_std(output_train).numpy():.4f}")
    
    # 推理模式
    output_inference = bn(x_test, training=False)
    print("\n推理模式:")
    print(f"  使用滑动均值: {bn.moving_mean.numpy()[:3]}...")
    print(f"  使用滑动方差: {bn.moving_variance.numpy()[:3]}...")

demonstrate_bn_behavior()

## 4. 完整训练示例

In [ ]:
# 加载 Fashion MNIST 数据集
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# 数据预处理
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

print(f"训练集: {X_train.shape}")
print(f"验证集: {X_valid.shape}")
print(f"测试集: {X_test.shape}")

In [ ]:
def create_model_with_bn(use_bn=True):
    """
    创建带或不带 BN 的模型用于对比
    
    Parameters:
    -----------
    use_bn : bool
        是否使用批量归一化
    
    Returns:
    --------
    keras.Model
        编译好的模型
    """
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    
    for units in [256, 128, 64]:
        model.add(keras.layers.Dense(units, kernel_initializer='he_normal', use_bias=not use_bn))
        if use_bn:
            model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation('elu'))
    
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 创建两个模型进行对比
model_with_bn = create_model_with_bn(use_bn=True)
model_without_bn = create_model_with_bn(use_bn=False)

print(f"带 BN 的模型参数量: {model_with_bn.count_params():,}")
print(f"不带 BN 的模型参数量: {model_without_bn.count_params():,}")

In [ ]:
# 训练对比（使用简化参数快速验证）
EPOCHS = 10
BATCH_SIZE = 64

print("训练带 BN 的模型...")
history_with_bn = model_with_bn.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_valid, y_valid),
    verbose=1
)

print("\n训练不带 BN 的模型...")
history_without_bn = model_without_bn.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_valid, y_valid),
    verbose=1
)

In [ ]:
# 绘制训练曲线对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 准确率对比
axes[0].plot(history_with_bn.history['accuracy'], 'b-', label='带 BN (训练)')
axes[0].plot(history_with_bn.history['val_accuracy'], 'b--', label='带 BN (验证)')
axes[0].plot(history_without_bn.history['accuracy'], 'r-', label='不带 BN (训练)')
axes[0].plot(history_without_bn.history['val_accuracy'], 'r--', label='不带 BN (验证)')
axes[0].set_title('准确率对比')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 损失对比
axes[1].plot(history_with_bn.history['loss'], 'b-', label='带 BN (训练)')
axes[1].plot(history_with_bn.history['val_loss'], 'b--', label='带 BN (验证)')
axes[1].plot(history_without_bn.history['loss'], 'r-', label='不带 BN (训练)')
axes[1].plot(history_without_bn.history['val_loss'], 'r--', label='不带 BN (验证)')
axes[1].set_title('损失对比')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('batch_normalization_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# 测试集评估
print("\n测试集评估:")
test_loss_bn, test_acc_bn = model_with_bn.evaluate(X_test, y_test, verbose=0)
test_loss_no_bn, test_acc_no_bn = model_without_bn.evaluate(X_test, y_test, verbose=0)

print(f"带 BN: 准确率 = {test_acc_bn:.4f}, 损失 = {test_loss_bn:.4f}")
print(f"不带 BN: 准确率 = {test_acc_no_bn:.4f}, 损失 = {test_loss_no_bn:.4f}")

## 5. BN 的超参数配置

In [ ]:
# BatchNormalization 的完整参数
bn_layer_custom = keras.layers.BatchNormalization(
    axis=-1,                 # 归一化的轴，-1 表示最后一个轴（特征轴）
    momentum=0.99,           # 滑动平均的动量（默认 0.99）
    epsilon=1e-3,            # 防止除零的小常数（默认 0.001）
    center=True,             # 是否添加 beta（偏移量）
    scale=True,              # 是否添加 gamma（缩放因子）
    beta_initializer='zeros',
    gamma_initializer='ones',
    moving_mean_initializer='zeros',
    moving_variance_initializer='ones'
)

print("BatchNormalization 超参数说明:")
print("="*50)
print("momentum: 滑动平均动量，值越大历史信息保留越多")
print("  - 0.99: 适合大数据集（默认）")
print("  - 0.9: 适合小数据集")
print("epsilon: 数值稳定性常数，通常无需修改")
print("center/scale: 是否使用 beta/gamma 参数")

## 6. 使用建议

### 何时使用 BN

| 场景 | 建议 |
|------|------|
| 深层网络 | 强烈推荐 |
| 训练不稳定 | 推荐 |
| 想用更大学习率 | 推荐 |
| 小 batch size (<16) | 考虑使用 Layer Normalization |
| RNN/LSTM | 使用 Layer Normalization |

### 注意事项

1. **Batch Size 影响**: 小 batch 时 BN 效果不稳定，考虑使用 Layer Normalization
2. **Dropout + BN**: 通常不建议同时使用，或将 Dropout 放在 BN 之后
3. **迁移学习**: 微调时可能需要冻结 BN 层的统计量
4. **推理模式**: 确保推理时 `training=False`

In [ ]:
# 验证代码正确性
print("批量归一化模块测试完成")
print("\n关键要点:")
print("1. BN 归一化每层输入，加速训练并稳定梯度")
print("2. 推荐在激活函数前使用 BN，并设置 use_bias=False")
print("3. 训练和推理阶段使用不同的统计量")
print("4. 小 batch size 时考虑使用 Layer Normalization")